## **2. Prétraitement**
- Segmentation (phrases)
- Tokenization (mots)
- Étiquetage morphosyntaxique (POS Tagging) 
- (Lemmatisation)
- Filtrage (stopwords)
- Extraction de termes complexes (MWE / n-grammes / segments répétés)
- Chunking / Filtrage par patrons syntaxiques (basés sur les patrons fréquents dans les MeSH)
- Extraction de collocations significatives (en fonction du Log-likelihood ratio)
- Extraction de concordances (KWIC) pour un ensemble de mots-clés d'intérêt
- Extraction de termes MeSH présents dans les données

### **Lire le corpus** 

In [138]:
lng = 'fr'
acteur = 'msss'
tag = ''
if tag:
    file = acteur + '/' + acteur + '_' + tag + '.csv'

else:
    if lng == 'fr':
        file = acteur +'.csv'
    if lng == 'en':
        file = acteur + '_en.csv'

In [139]:
import shutil, re, random
from os import listdir, chdir, path
from pathlib import Path
from pandas import *
import glob

import nltk
#nltk.download(['popular'])
from nltk.tokenize import RegexpTokenizer
tokenizer_re = RegexpTokenizer(r"\w\'|\w+")
from nltk import bigrams, trigrams, ngrams, everygrams
from nltk.probability import FreqDist


import treetaggerwrapper
tagger = treetaggerwrapper.TreeTagger(TAGLANG=lng)


from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import binom, chi2

In [168]:
def lire_corpus(acteur = acteur, langue=lng):
    base_path = '../03-corpus/2-data/'
    
    folder_path = path.join(base_path, '1-' + langue, acteur)
    all_files = glob.glob(path.join(folder_path, "*.csv"))
    tags = [f.split('_')[1][:-4] for f in listdir(folder_path)]

    df = DataFrame()
    for f, tag in zip(all_files, tags):
        csv = read_csv(f, encoding='utf-8', sep=',')
        csv = csv[~csv["Address"].str.contains('pdf')] #  Problèmes 
        #csv = csv['text']
        # Using DataFrame.insert() to add a column
        csv.insert(1, "Corpus", acteur)
        csv.insert(2, "Sous-corpus", tag)
        df = concat([df, csv]) [['Corpus', 'Sous-corpus', 'Title', 'text']]
    return df

In [169]:
data = lire_corpus(acteur) 
nb_docs = len(data)
print("On a un corpus de {} documents.".format(nb_docs))

On a un corpus de 0 documents.


In [142]:
data

,Corpus,Sous-corpus,text
0,msss,aines,Forum des partenaires pour contrer la maltrait...
1,msss,aines,Journée annuelle de concertation sur la proche...
2,msss,aines,Personnes âgées À propos Les personnes âgées d...
3,msss,aines,Plan d'action gouvernemental pour contrer la m...
4,msss,aines,"Politique Vieillir et vivre ensemble, chez soi..."
...,...,...,...
81,msss,zoonoses,Maladie de Lyme Traitement La maladie de Lyme ...
82,msss,zoonoses,Maladie de Lyme Formation sur la démarche clin...
83,msss,zoonoses,Maladie de Lyme Déclarer un cas de maladie de ...
84,msss,zoonoses,Maladie de Lyme Sites d'intérêt Maladie de Lym...


### **Nettoyage**

In [143]:
punct = '[!#$%&•►*+,;\/\\<=>?@[\]^_{|}~©«»—“”–—]'
spaces = '\s+'
postals = '([a-zA-Z]+\d+|\d+[a-zA-Z]+)+'
# phones = '\d{3}\s\d{3}-\d{4}' #très simple (trop)

text = [str(t).strip('\n').lower().replace('’', '\'') for t in data['text'].tolist()]
text = [re.sub(spaces, ' ', t) for t in text]
text = [re.sub(postals, ' STOP ', t) for t in text]
text = [re.sub(punct, ' STOP ', t) for t in text]
text = [t.replace("  ", " " ) for t in text]

### **Extraire un échantillon aléatoire**

Sinon, on n'arrive pas à traiter la totalité du corpus pour des raisons de performance

In [145]:
def sample_corpus(corpus, ratio):
    n = round(ratio * len(corpus))
    corpus = random.sample(text, n)
    print("On va travailler sur un échantillon correspondant à environ " + str(ratio * 100) + " % des documents du corpus, soit {} documents". format(len(corpus)))
    return " ".join(corpus)
    
corpus = sample_corpus(text, 1)

On va travailler sur un échantillon correspondant à environ 100 % des documents du corpus, soit 6116 documents


### **Filtrage (MWE - stopwords formés de plusieurs tokens)**
Surtout pour filtrer les expressions relatives à l'architecture d'information / navigation Web

In [146]:
def filter_mwesw(corpus):
    file_mwesw = '../04-filtrage/mwe_stopwords.txt'
    with open (file_mwesw, 'r', encoding='utf-8') as f:
        mwe_sw = [t.lower().strip('\n') for t in f.readlines()]
    for mwe in mwe_sw:
        corpus = corpus.replace(mwe, ' STOP ').replace('  ', " ")
    return corpus

In [147]:
corpus = filter_mwesw(corpus)

### **Tokenisation / POS tagging** (TreeTagger)  
https://github.com/miotto/treetagger-python/blob/master/README.rst  
https://treetaggerwrapper.readthedocs.io/en/latest/

In [148]:
# Ici, on tokenise une première fois avec le Regex Tokenizer de NLTK pour voir combien de temps ça devrait 
# prendre au Tree Tagger pour tokeniser et tagger notre corpus
def tok(corpus):
    # Seulement les caractères alphabétiques
    tokens = tokenizer_re.tokenize(corpus)
    print("Avec le RegExpTokenizer, notre corpus contient {} tokens.".format(len(tokens)))
    temps = round(len(tokens) / 15000 / 60)
    print('Le POS tagging devrait prendre environ {} minutes.'.format(temps))

temps = tok(corpus)

Avec le RegExpTokenizer, notre corpus contient 4278180 tokens.
Le POS tagging devrait prendre environ 5 minutes.


In [149]:
def tagging(corpus):
    output = []
    for t in tagger.tag_text(corpus):
        try: 
            output.append((t.split('\t')[0], t.split('\t')[1]))
        except Exception as e:
            output.append(('STOP', 'NAM'))

    return output


In [150]:
tagged = tagging(corpus)
tokens = [t[0] for t in tagged]

In [151]:
tagged

[('vaccinologie', 'NOM'),
 ('pratique', 'ADJ'),
 ('troubles', 'ADJ'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('coagulation', 'NOM'),
 ('les', 'DET:ART'),
 ('personnes', 'NOM'),
 ('qui', 'PRO:REL'),
 ('sont', 'VER:pres'),
 ('atteintes', 'VER:pper'),
 ('de', 'PRP'),
 ('troubles', 'NOM'),
 ('de', 'PRP'),
 ('la', 'DET:ART'),
 ('coagulation', 'NOM'),
 ('ou', 'KON'),
 ('de', 'PRP'),
 ('thrombocytopénie', 'NOM'),
 ('grave', 'ADJ'),
 ('STOP', 'NOM'),
 ('ou', 'KON'),
 ('qui', 'PRO:REL'),
 ('sont', 'VER:pres'),
 ('sous', 'PRP'),
 ('anticoagulothérapie', 'NOM'),
 ('STOP', 'NOM'),
 ('peuvent', 'VER:pres'),
 ('présenter', 'VER:infi'),
 ('des', 'PRP:det'),
 ('hématomes', 'NOM'),
 ('au', 'PRP:det'),
 ('point', 'NOM'),
 ("d'", 'PRP'),
 ('une', 'DET:ART'),
 ('injection', 'NOM'),
 ('.', 'SENT'),
 ('chez', 'PRP'),
 ('les', 'DET:ART'),
 ('personnes', 'NOM'),
 ('recevant', 'VER:ppre'),
 ('des', 'PRP:det'),
 ('facteurs', 'NOM'),
 ('de', 'PRP'),
 ('remplacement', 'NOM'),
 ('STOP', 'NOM'),
 ('le', 'DET:ART'),
 

In [152]:
# Si on veut lemmatiser
### **Mapping POS Tags** (FRMG)

#Pour utiliser adéquatement notre lemmatiseur par la suite (FrenchLefffLemmatizer), on va mapper les étiquettes morphosyntaxiques du TreeTagger à celles que prend le lemmatiseur (celles issues de FRMG)

#http://alpage.inria.fr/frmgwiki/content/tagset-frmg

# file_path = '../04-filtrage/mapping_treeTagger_lefff.csv'

# with open(file_path) as f:
#    csv = read_csv(f)

# treeTag = [term for term in csv['TreeTagger'].tolist()] 
# lefff = [term for term in csv['Lefff'].tolist()]

# mapping = {term : lefff[treeTag.index(term)] for term in treeTag}

In [153]:
#tagged_lem = [[t.split('\t')[0], mapping[t.split('\t')[1]]] for t in tagger.tag_text(corpus)]

In [154]:
# Si on veut lemmatiser
### **Lemmatisation** (FrenchLefffLemmatizer)
#https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer

# from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
# lemmatizer = FrenchLefffLemmatizer()

# lemmas = []
# for term in tagged_lem:
#     term_l = []
#     if lemmatizer.lemmatize(term[0], term[1]) == []:
#        term_l = (lemmatizer.lemmatize(term[0]), term[1])
    
#     elif type(lemmatizer.lemmatize(term[0], term[1])) == str:
#         term_l  = (lemmatizer.lemmatize(term[0], term[1]), term[1])

#     else:
#         term_l = tuple(lemmatizer.lemmatize(term[0], term[1])[0])

#     lemmas.append(term_l)

### **Collocations / Phrases / N-Grammes (MWE)**
https://www.kaggle.com/code/alvations/n-gram-language-model-with-nltk/notebook  

In [155]:
def extr_ngrams(tagged):
    ngrammes= list(everygrams(tagged, min_len=1, max_len=10))
    print("Avant filtrage, on a {} ngrammes.".format(len(ngrammes)))
    return ngrammes

In [156]:
ngrammes = extr_ngrams(tagged)
#ngrammes_lem = extr_ngrams(tagged_lem)

Avant filtrage, on a 44495595 ngrammes.


In [157]:
ngrammes

[(('vaccinologie', 'NOM'),),
 (('vaccinologie', 'NOM'), ('pratique', 'ADJ')),
 (('vaccinologie', 'NOM'), ('pratique', 'ADJ'), ('troubles', 'ADJ')),
 (('vaccinologie', 'NOM'),
  ('pratique', 'ADJ'),
  ('troubles', 'ADJ'),
  ('de', 'PRP')),
 (('vaccinologie', 'NOM'),
  ('pratique', 'ADJ'),
  ('troubles', 'ADJ'),
  ('de', 'PRP'),
  ('la', 'DET:ART')),
 (('vaccinologie', 'NOM'),
  ('pratique', 'ADJ'),
  ('troubles', 'ADJ'),
  ('de', 'PRP'),
  ('la', 'DET:ART'),
  ('coagulation', 'NOM')),
 (('vaccinologie', 'NOM'),
  ('pratique', 'ADJ'),
  ('troubles', 'ADJ'),
  ('de', 'PRP'),
  ('la', 'DET:ART'),
  ('coagulation', 'NOM'),
  ('les', 'DET:ART')),
 (('vaccinologie', 'NOM'),
  ('pratique', 'ADJ'),
  ('troubles', 'ADJ'),
  ('de', 'PRP'),
  ('la', 'DET:ART'),
  ('coagulation', 'NOM'),
  ('les', 'DET:ART'),
  ('personnes', 'NOM')),
 (('vaccinologie', 'NOM'),
  ('pratique', 'ADJ'),
  ('troubles', 'ADJ'),
  ('de', 'PRP'),
  ('la', 'DET:ART'),
  ('coagulation', 'NOM'),
  ('les', 'DET:ART'),
  ('pers

### **Extraction des patrons syntaxiques**

In [158]:
def extract_patterns(ngrammes):
    patterns = []
    for ng in ngrammes:
        phrase = tuple([t[0] for t in ng])
        pattern = [t[1] for t in ng]
        patterns.append([phrase, pattern])
    return patterns

In [159]:
phrases = extract_patterns(ngrammes)
# phrases_lem = extract_patterns(ngrammes_lem)

KeyboardInterrupt: 

In [ ]:
# def freq(phrases):
#     return FreqDist([" ".join(t[0]).replace("' ", "'") for t in phrases])

# def freq(phrases):
#     return FreqDist([t[0] for t in phrases])

In [ ]:
frequencies = FreqDist(everygrams(tokens, min_len=1, max_len=10))

In [ ]:
frequencies

FreqDist({('STOP',): 4248, ('de',): 2926, ('la',): 1999, ('.',): 1748, ('covid-19',): 1225, ('et',): 1132, ('des',): 1089, ('les',): 1032, ('à',): 940, ('le',): 809, ...})

### **Filtrage** 
On retire les n-grammes qui débutent ou se terminent par un stopword (antidictionnaire)

In [ ]:
# Importer l'antidictionnaire pour filtrer les données

# # Stopwords lemmatisés
# file_path = '../04-filtrage/stopwords_lemmatized.txt'
# with open(file_path, 'r', encoding="utf-8") as f:
#     stopwords_lemmatized = [w.strip('\n').lower() for w in f.readlines()]

# Stopwords fréquents en français (non lemmatisés)
file_path = "../04-filtrage/stopwords.txt"
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords = [t.lower().strip('\n') for t in f.readlines()]


# Stopwords fréquents en anglais (non lemmatisés)
file_path = '../04-filtrage/stop_words_english.txt'
with open(file_path, 'r', encoding="utf-8") as f:
    stopwords += [t.lower().strip('\n') for t in f.readlines()]

*LONG*

In [ ]:
def filtrer_stopwords(x): # On s'assure aussi que le premier terme du ngramme est un NOM pour avoir des syntagmes nominaux
    if lng == 'en':
        nom = 'NN'
    if lng == 'fr':
        nom = 'NOM'
    return [term for term in x if not 'STOP' in term[0] and not term[0][0] in stopwords and not term[0][-1] in stopwords \
        and not 'NUM' in term[1] and term[1][0] == nom and not '.' in term[0] and not '-' in term[0] and not ':' in term[0]\
        
        # Une parenthèse fermante peut juste se trouver comme dernier token
        # Si une parenthèse est ouverte, elle doit aussi être fermée (et vice versa)
        and not ')' in term[0][:-1] and not ('(' in term[0] and not ')' in term[0]) \
        and not (')' in term[0] and not '(' in term[0])]

In [ ]:
phrases = filtrer_stopwords(phrases)

In [ ]:
test = [t[0] for t in phrases]

test

[('prévention',),
 ('prévention', 'des', 'infections'),
 ('prévention', 'des', 'infections', 'en', 'services'),
 ('prévention', 'des', 'infections', 'en', 'services', 'de', 'garde'),
 ('prévention',
  'des',
  'infections',
  'en',
  'services',
  'de',
  'garde',
  'et',
  'écoles'),
 ('prévention',
  'des',
  'infections',
  'en',
  'services',
  'de',
  'garde',
  'et',
  'écoles',
  'covid-19'),
 ('infections',),
 ('infections', 'en', 'services'),
 ('infections', 'en', 'services', 'de', 'garde'),
 ('infections', 'en', 'services', 'de', 'garde', 'et', 'écoles'),
 ('infections', 'en', 'services', 'de', 'garde', 'et', 'écoles', 'covid-19'),
 ('services',),
 ('services', 'de', 'garde'),
 ('services', 'de', 'garde', 'et', 'écoles'),
 ('services', 'de', 'garde', 'et', 'écoles', 'covid-19'),
 ('garde',),
 ('garde', 'et', 'écoles'),
 ('garde', 'et', 'écoles', 'covid-19'),
 ('écoles',),
 ('écoles', 'covid-19'),
 ('intervenants',),
 ('intervenants', 'pour', 'les', 'services'),
 ('intervenant

On retire les n-grammes qui débutent ou se terminent par un token dont la longueur est inférieure à 2 caractères ou supérieure à 18 caractères

In [ ]:
def filter_len(x):
    return [term for term in x if \
        (len(term[0][0]) > 2 or term[0][0] == '(')  and (len(term[0][-1]) > 2 or term[0][-1] == ')') and \
        len(term[0][0]) < 18 and len(term[0][-1]) < 18]

In [ ]:
phrases = filter_len(phrases)

In [ ]:
phrases

[[('prévention',), ['NOM']],
 [('prévention', 'des', 'infections'), ['NOM', 'PRP:det', 'NOM']],
 [('prévention', 'des', 'infections', 'en', 'services'),
  ['NOM', 'PRP:det', 'NOM', 'PRP', 'NOM']],
 [('prévention', 'des', 'infections', 'en', 'services', 'de', 'garde'),
  ['NOM', 'PRP:det', 'NOM', 'PRP', 'NOM', 'PRP', 'NOM']],
 [('prévention',
   'des',
   'infections',
   'en',
   'services',
   'de',
   'garde',
   'et',
   'écoles'),
  ['NOM', 'PRP:det', 'NOM', 'PRP', 'NOM', 'PRP', 'NOM', 'KON', 'NOM']],
 [('prévention',
   'des',
   'infections',
   'en',
   'services',
   'de',
   'garde',
   'et',
   'écoles',
   'covid-19'),
  ['NOM', 'PRP:det', 'NOM', 'PRP', 'NOM', 'PRP', 'NOM', 'KON', 'NOM', 'ADJ']],
 [('infections',), ['NOM']],
 [('infections', 'en', 'services'), ['NOM', 'PRP', 'NOM']],
 [('infections', 'en', 'services', 'de', 'garde'),
  ['NOM', 'PRP', 'NOM', 'PRP', 'NOM']],
 [('infections', 'en', 'services', 'de', 'garde', 'et', 'écoles'),
  ['NOM', 'PRP', 'NOM', 'PRP', 'NOM'

On retire les n-grammes qui apparaissent moins de 15 fois dans le corpus

In [ ]:
def filter_freq(x):
    return [term for term in x if frequencies[tuple(term[0])] > 15]

In [ ]:
phrases = filter_freq(phrases)

In [ ]:
phrases

[[('prévention',), ['NOM']],
 [('infections',), ['NOM']],
 [('services',), ['NOM']],
 [('services',), ['NOM']],
 [('coronavirus',), ['NOM']],
 [('coronavirus', '(', 'covid-19', ')'), ['NOM', 'PUN', 'NOM', 'PUN']],
 [('covid-19',), ['NOM']],
 [('services',), ['NOM']],
 [('milieux',), ['NOM']],
 [('milieu',), ['NOM']],
 [('covid-19',), ['NOM']],
 [('informations',), ['NOM']],
 [('covid-19',), ['NOM']],
 [('santé',), ['NOM']],
 [('santé', 'publique'), ['NOM', 'ADJ']],
 [('québec',), ['NOM']],
 [('inspq',), ['NOM']],
 [('covid-19',), ['NOM']],
 [('santé',), ['NOM']],
 [('santé', 'et', 'des', 'services'), ['NOM', 'KON', 'PRP:det', 'NOM']],
 [('santé', 'et', 'des', 'services', 'sociaux'),
  ['NOM', 'KON', 'PRP:det', 'NOM', 'ADJ']],
 [('services',), ['NOM']],
 [('services', 'sociaux'), ['NOM', 'ADJ']],
 [('covid-19',), ['NOM']],
 [('covid-19',), ['NOM']],
 [('covid-19', 'directives'), ['NOM', 'ADJ']],
 [('mois',), ['NOM']],
 [('isolement',), ['NOM']],
 [('travailleurs',), ['NOM']],
 [('travai

In [ ]:
#phrases = [[" ".join(term[0]).replace("' ", "'"), " ".join(term[1])] for term in phrases]
phrases = [[term[0], " ".join(term[1])] for term in phrases]
# phrases_lemmatized = filtrer_phrases(phrases_lemmatized, freq_lemmatized)

In [ ]:
for phrase in phrases:
    phrase.append(frequencies[tuple(phrase[0])])

In [ ]:
print("Après filtrage, on a {} occurrences de ngrammes.".format(len(phrases))) 

Après filtrage, on a 11104 occurrences de ngrammes.


In [ ]:
phrases[:15]

[[('prévention',), 'NOM', 42],
 [('infections',), 'NOM', 53],
 [('services',), 'NOM', 156],
 [('services',), 'NOM', 156],
 [('coronavirus',), 'NOM', 71],
 [('coronavirus', '(', 'covid-19', ')'), 'NOM PUN NOM PUN', 63],
 [('covid-19',), 'NOM', 1225],
 [('services',), 'NOM', 156],
 [('milieux',), 'NOM', 82],
 [('milieu',), 'NOM', 42],
 [('covid-19',), 'NOM', 1225],
 [('informations',), 'NOM', 37],
 [('covid-19',), 'NOM', 1225],
 [('santé',), 'NOM', 243],
 [('santé', 'publique'), 'NOM ADJ', 51]]

In [ ]:
DataFrame(phrases, columns=["Expression", "Patron syntaxique", "Fréquence"])

phrases

[[('prévention',), 'NOM', 42],
 [('infections',), 'NOM', 53],
 [('services',), 'NOM', 156],
 [('services',), 'NOM', 156],
 [('coronavirus',), 'NOM', 71],
 [('coronavirus', '(', 'covid-19', ')'), 'NOM PUN NOM PUN', 63],
 [('covid-19',), 'NOM', 1225],
 [('services',), 'NOM', 156],
 [('milieux',), 'NOM', 82],
 [('milieu',), 'NOM', 42],
 [('covid-19',), 'NOM', 1225],
 [('informations',), 'NOM', 37],
 [('covid-19',), 'NOM', 1225],
 [('santé',), 'NOM', 243],
 [('santé', 'publique'), 'NOM ADJ', 51],
 [('québec',), 'NOM', 98],
 [('inspq',), 'NOM', 23],
 [('covid-19',), 'NOM', 1225],
 [('santé',), 'NOM', 243],
 [('santé', 'et', 'des', 'services'), 'NOM KON PRP:det NOM', 40],
 [('santé', 'et', 'des', 'services', 'sociaux'),
  'NOM KON PRP:det NOM ADJ',
  40],
 [('services',), 'NOM', 156],
 [('services', 'sociaux'), 'NOM ADJ', 61],
 [('covid-19',), 'NOM', 1225],
 [('covid-19',), 'NOM', 1225],
 [('covid-19', 'directives'), 'NOM ADJ', 19],
 [('mois',), 'NOM', 71],
 [('isolement',), 'NOM', 21],
 [('

In [ ]:
print("Après filtrage, on a {} ngrammes uniques.".format(len(DataFrame(phrases).drop_duplicates())))

Après filtrage, on a 285 ngrammes uniques.


### **Filtrage (Patrons syntaxiques)**  
Lossio-Ventura, J. A., Jonquet, C., Roche, M., & Teisseire, M. (2014). Biomedical Terminology Extraction : A new combination of Statistical and Web Mining Approaches. 421. https://hal-lirmm.ccsd.cnrs.fr/lirmm-01056598

On veut aller extraire les structures syntaxiques les plus courantes dans les MeSH pour filtrer notre corpus selon celles-ci (inspiré de la méthodologie de l'article ci-dessus ; voir le Notebook *Mesh_extract.ipynb*). Pour ce faire, nous allons donc ne sélectionner que les ngrammes qui y correspondent. 

In [ ]:
file_patterns = '../04-filtrage/MeSH/mesh_patterns-fr.csv'

with open (file_patterns, 'r') as f:
    patterns = read_csv(f)
    patterns = patterns['Structure'].tolist() #[:200] # Pour prendre les 200 structures syntaxiques les plus fréquentes dans les MeSH

In [ ]:
def filter_patterns(phrases):
    return [t for t in phrases if t[1] in patterns and not 'NUM' in t[1]] # and not 'NOM NOM' in t[1]
# terms_lemmatized = [t for t in phrases_lemmatized if t[1] in patterns]

In [ ]:
terms = filter_patterns(phrases)
#terms = phrases

In [ ]:
terms

[[('prévention',), 'NOM', 42],
 [('infections',), 'NOM', 53],
 [('services',), 'NOM', 156],
 [('services',), 'NOM', 156],
 [('coronavirus',), 'NOM', 71],
 [('coronavirus', '(', 'covid-19', ')'), 'NOM PUN NOM PUN', 63],
 [('covid-19',), 'NOM', 1225],
 [('services',), 'NOM', 156],
 [('milieux',), 'NOM', 82],
 [('milieu',), 'NOM', 42],
 [('covid-19',), 'NOM', 1225],
 [('informations',), 'NOM', 37],
 [('covid-19',), 'NOM', 1225],
 [('santé',), 'NOM', 243],
 [('santé', 'publique'), 'NOM ADJ', 51],
 [('québec',), 'NOM', 98],
 [('inspq',), 'NOM', 23],
 [('covid-19',), 'NOM', 1225],
 [('santé',), 'NOM', 243],
 [('services',), 'NOM', 156],
 [('services', 'sociaux'), 'NOM ADJ', 61],
 [('covid-19',), 'NOM', 1225],
 [('covid-19',), 'NOM', 1225],
 [('covid-19', 'directives'), 'NOM ADJ', 19],
 [('mois',), 'NOM', 71],
 [('isolement',), 'NOM', 21],
 [('travailleurs',), 'NOM', 35],
 [('travailleurs', 'de', 'la', 'santé'), 'NOM PRP DET:ART NOM', 27],
 [('santé',), 'NOM', 243],
 [('services',), 'NOM', 15

In [ ]:
print("Le filtrage syntaxique élimine environ {} % des termes".format(round((len(phrases) - len(terms)) / len(phrases) * 100)))
print("On avait {} ngrammes, ".format(len(phrases)) + "on en a maintenant {}.".format(len(terms)))

Le filtrage syntaxique élimine environ 2 % des termes
On avait 11104 ngrammes, on en a maintenant 10868.


In [ ]:
# import pandas as pd

# def extract_terms(liste_terms):
#     file_path = '../04-filtrage/output/'
#     tab = pd.DataFrame(terms, columns= ["Expression", "Structure syntaxique", "Fréquence"]).drop_duplicates(subset='Expression', keep="last")
#     tab.sort_values(["Fréquence"], 
#                         axis=0,
#                         ascending=[False], 
#                         inplace=True)

#     return 

#     # file_path = path.join(file_path, tag, tag)                    
#     # tab.to_csv(file_path + '_terms.csv')

# extract_terms(terms)
# #extract_terms(terms_lemmatized)

In [ ]:
for phrase in terms:
    phrase[0] = tuple(phrase[0])


terms_patterns = DataFrame(terms, columns = ["Expression", "Structure syntaxique", "Fréquence"])
terms_patterns = terms_patterns.to_dict('records')
dict_patterns = {}
for term in terms_patterns:
     exp = term['Expression']
     pattern = term['Structure syntaxique']
     dict_patterns[exp] = pattern

In [ ]:
terms_patterns

[{'Expression': ('prévention',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 42},
 {'Expression': ('infections',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 53},
 {'Expression': ('services',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 156},
 {'Expression': ('services',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 156},
 {'Expression': ('coronavirus',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 71},
 {'Expression': ('coronavirus', '(', 'covid-19', ')'),
  'Structure syntaxique': 'NOM PUN NOM PUN',
  'Fréquence': 63},
 {'Expression': ('covid-19',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1225},
 {'Expression': ('services',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 156},
 {'Expression': ('milieux',), 'Structure syntaxique': 'NOM', 'Fréquence': 82},
 {'Expression': ('milieu',), 'Structure syntaxique': 'NOM', 'Fréquence': 42},
 {'Expression': ('covid-19',),
  'Structure syntaxique': 'NOM',
  'Fréquence': 1225},
 {'Expression': ('informations',),
  

### **Filtrage (Collocations statistiquement significatives)** Log-Likelihood Ratio

[Notebook - Collocation extraction methodologies compared](https://notebooks.githubusercontent.com/view/ipynb?azure_maps_enabled=false&browser=chrome&color_mode=auto&commit=33868e847376764d7733cd958986c88dedfaec97&device=unknown&enc_url=68747470733a2f2f7261772e67697468756275736572636f6e74656e742e636f6d2f746f64642d636f6f6b2f4d4c2d596f752d43616e2d5573652f333338363865383437333736373634643737333363643935383938366338386465646661656339372f70726f626162696c69737469635f6c616e67756167655f6d6f64656c696e672f636f6c6c6f636174696f6e5f65787472616374696f6e732e6970796e62&enterprise_enabled=false&logged_in=false&nwo=todd-cook%2FML-You-Can-Use&path=probabilistic_language_modeling%2Fcollocation_extractions.ipynb&platform=android&repository_id=167140788&repository_type=Repository&version=102)

On applique un test d'hypothèse statistique aux n-grammes sur lesquels une probabilité a été mesurée (Log-likelihood ratio) - seuls les n-grammes dont le test est significatif seront conservés.
On considère que l'apparition de ces collocations dans notre corpus n'est pas dûe au hasard.

In [ ]:
def loglikelihood_ratio(c_prior, c_n, c_ngram, N):
    """
    Compute the ratio of two hypotheses of likelihood and return the ratio.
    The formula here and test verification values are taken from 
    Manning & Schūtze _Foundations of Statistical Natural Language Processing_ p.172-175
    Parameters:
    c_prior: count of word 1 if bigrams or count of [w1w2 .. w(n-1)] if ngram
    c_n : count of word 2 if bigrams or count of wn if ngram
    c12: count of bigram (w1, w2) if bigram or count of ngram if ngram
    N: the number of words in the corpus
    """

    p = c_n / N
    p1 = c_ngram / c_prior
    p2 = (c_n - c_ngram) / (N - c_prior)   
    # We proactively trap a runtimeWarning: divide by zero encountered in log,
    # which may occur with extreme collocations
    import warnings
    with warnings.catch_warnings(): # this will reset our filterwarnings setting
        warnings.filterwarnings('error')
        try:
            return (np.log(binom.pmf(c_ngram, c_prior, p)) 
                    + np.log(binom.pmf(c_n - c_ngram, N - c_prior, p)) 
                    - np.log(binom.pmf(c_ngram, c_prior, p1) )
                    - np.log(binom.pmf(c_n - c_ngram, N - c_prior, p2)))             
        except Warning:
            return np.inf 

In [ ]:
len_prior = len(terms)
print("Au départ, on a {} ngrammes.".format(len_prior))

Au départ, on a 10868 ngrammes.


In [ ]:
frequencies

FreqDist({('STOP',): 4248, ('de',): 2926, ('la',): 1999, ('.',): 1748, ('covid-19',): 1225, ('et',): 1132, ('des',): 1089, ('les',): 1032, ('à',): 940, ('le',): 809, ...})

In [ ]:
# Pour le calcul des probabilités, on a besoin de traiter séparément les ngrammes selon la valeur de n
N = len(tokens)
fd_tokens = nltk.FreqDist(tokens)

In [ ]:
def llr_ngrammes(terms):
    llr = []

    ngrammes = [term[0] for term in terms]
        
    for t in ngrammes:
        if len(t) == 1:
            try:
                llr.append({'Terme' : str(t[0]), 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : fd_tokens[str(t[0])], 'LLR': '-', 'p-value': '-'})
            except Exception as e:
                print(t, str(e))
        else:
            c_prior = frequencies[t[:-1]] # Antécédent = P(w1w2..w_n-1) (si on considère que P(w1w2...wn) = P(wn) | P(w1w2...w_n-1)
            c_n = fd_tokens[t[-1]]     # Dernier mot du ngramme  P(wn)
            c_ngram = frequencies[t]             # Le ngramme lui-même P(w1w2w3..wn)

            try:
                res = -2 * loglikelihood_ratio(c_prior, c_n, c_ngram, N)
                p_value = chi2.sf(res, 1) # 1 degrees of freedom
                #if res == float('-inf') :
                #    res = 50000

                if p_value < 0.01 or (res == float('-inf')):
                    #llr.append({'Collocation' : " ".join(t).replace("' ", "'").replace("( ", "(").replace(" )", ")"), 'Structure syntaxique': dict_patterns[" ".join(t).replace("' ", "'")], 'Fréquence' : c_ngram, 'LLR': res, 'p-value': p_value})
                    llr.append({'Terme' : t, 'Structure syntaxique': dict_patterns[t], 'Fréquence (TF)' : c_ngram, 'LLR': res, 'p-value': p_value})
            
            except Exception as e:
                print(t, str(e))
            
    return llr


In [ ]:
significant_coll = llr_ngrammes(terms)
#terms = [{'Collocation' : t[0], 'Structure syntaxique': t[1], 'Fréquence' : t[2]} for t in terms] # Si on veut pas appliquer le LLR

In [ ]:
significant_coll


[{'Terme': 'prévention',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 42,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'infections',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 53,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'services',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 156,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'services',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 156,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'coronavirus',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 71,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': ('coronavirus', '(', 'covid-19', ')'),
  'Structure syntaxique': 'NOM PUN NOM PUN',
  'Fréquence (TF)': 63,
  'LLR': 561.9998126233246,
  'p-value': 3.087407047360579e-124},
 {'Terme': 'covid-19',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 1225,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': 'services',
  'Structure syntaxique': 'NOM',
  'Fréquence (TF)': 156,
  'LLR': '-',
  'p-value': '-'},
 {'Terme': '

In [ ]:
print('Après avoir calculé le log-likelihood ratio, on a retiré {} collocations qui n\'étaient pas statistiquement significatives.'.format(len(terms) - len(significant_coll)))
print('Ça représente environ {} % de nos n-grammes.'.format(round((len(terms) - len(significant_coll)) / len(terms) *100 )))

Après avoir calculé le log-likelihood ratio, on a retiré 0 collocations qui n'étaient pas statistiquement significatives.
Ça représente environ 0 % de nos n-grammes.


In [ ]:
df = DataFrame(significant_coll).sort_values(by = "Fréquence (TF)", ascending=False).drop_duplicates()

# On veut faire join pour tous les termes[collocation]
def join_term(x):
    if type(x) == tuple:
        return " ".join(x).replace("' ", "'").replace("( ", "(").replace(" )", ")")
    else:
        return x

df['Terme'] = df['Terme'].apply(lambda x: join_term(x))

if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 

df.to_csv(output_path)

df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value
3053,covid-19,NOM,1225,-,-
3404,directive,NOM,501,-,-
2218,dose,NOM,470,-,-
4682,vaccin,NOM,385,-,-
1704,vaccination,NOM,252,-,-
...,...,...,...,...,...
7300,rougeur,NOM,16,-,-
3798,services sociaux directive,NOM ADJ ADJ,16,83.770985,0.0
3801,orientations,NOM,16,-,-
8678,femmes,NOM,16,-,-


### **Filtrage - Fréquence documentaire**
** Il y aurait quelque chose à modifier ici ; on a une fréquence documentaire de 0 pour les ngrammes qui n'ont plus la forme exacte qu'ils avaient dans le corpus orginal. **

In [ ]:
dfs = {term: len([doc for doc in text if term in doc]) for term in df['Terme'].tolist()}

In [ ]:
max_df = round(0.98 * nb_docs)        # Pour rejeter les termes qui se retrouvent dans plus de 98% des documents 
max_df

77

In [ ]:
max_df_observed = dfs[max(dfs, key=dfs.get)] # Voir quelle est la fréquence documentaire maximale qu'on retrouve
max_df_observed

79

In [ ]:
max_df_to_remove = {term:df for term,df in dfs.items() if df > max_df}
max_df_to_remove

{'covid-19': 79}

In [ ]:
dfs = {term:df for term,df in dfs.items() if df < max_df and df > 0} # Si df = 0, c'est un artefact créé par le prétraitement lui-même

Porter attention aux termes qui sortent ici - en comptant la fréquence documentaire, techniquement on ne devrait pas retrouver de 0

In [ ]:
zeros =  {term:df for term,df in dfs.items() if df == 0}  
print(len(zeros))
zeros 

0


{}

In [ ]:
dfs = DataFrame(list(dfs.items()),columns = ['Terme','Fréquence documentaire (DF)']) 

dfs

,Terme,Fréquence documentaire (DF)
0,directive,40
1,dose,16
2,vaccin,40
3,vaccination,39
4,santé,43
...,...,...
255,rougeur,2
256,services sociaux directive,4
257,orientations,10
258,femmes,10


In [ ]:
df = df.merge(dfs, on='Terme').drop_duplicates()

df.sort_values(['Fréquence (TF)'], 
            axis=0,
            ascending=[False], 
            inplace=True)

In [ ]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF)
0,directive,NOM,501,-,-,40
1,dose,NOM,470,-,-,16
2,vaccin,NOM,385,-,-,40
3,vaccination,NOM,252,-,-,39
4,santé,NOM,243,-,-,43
...,...,...,...,...,...,...
248,rsss,NOM,16,-,-,9
247,sodium,NOM,16,-,-,3
246,fréquence,NOM,16,-,-,9
245,trajectoire,NOM,16,-,-,12


In [ ]:
list_terms = df['Terme'].tolist()

### **KWIC (Keyword in Context)**
Termes d'intérêt : 
- « Programme »
- « Plan »
- « Service(s) de » 
- « Intervenant(e) en »
- « Professionnel de »
- « Institut (du/de) »
- « Groupe de recherche en »
- « Personne »
- « Infirmière (en) »

In [ ]:
ngrammes

[(('prévention', 'NOM'),),
 (('prévention', 'NOM'), ('des', 'PRP:det')),
 (('prévention', 'NOM'), ('des', 'PRP:det'), ('infections', 'NOM')),
 (('prévention', 'NOM'),
  ('des', 'PRP:det'),
  ('infections', 'NOM'),
  ('en', 'PRP')),
 (('prévention', 'NOM'),
  ('des', 'PRP:det'),
  ('infections', 'NOM'),
  ('en', 'PRP'),
  ('services', 'NOM')),
 (('prévention', 'NOM'),
  ('des', 'PRP:det'),
  ('infections', 'NOM'),
  ('en', 'PRP'),
  ('services', 'NOM'),
  ('de', 'PRP')),
 (('prévention', 'NOM'),
  ('des', 'PRP:det'),
  ('infections', 'NOM'),
  ('en', 'PRP'),
  ('services', 'NOM'),
  ('de', 'PRP'),
  ('garde', 'NOM')),
 (('prévention', 'NOM'),
  ('des', 'PRP:det'),
  ('infections', 'NOM'),
  ('en', 'PRP'),
  ('services', 'NOM'),
  ('de', 'PRP'),
  ('garde', 'NOM'),
  ('et', 'KON')),
 (('prévention', 'NOM'),
  ('des', 'PRP:det'),
  ('infections', 'NOM'),
  ('en', 'PRP'),
  ('services', 'NOM'),
  ('de', 'PRP'),
  ('garde', 'NOM'),
  ('et', 'KON'),
  ('écoles', 'NOM')),
 (('prévention', 'NO

In [ ]:
# Dans notre cas on veut que ça débute par le mot-clé donc le contexte est un peu plus simple
# penser à généraliser avec des expressions régulières
kw = ['programme', 'plan ', 'service', 'intervenant', 'infirmière en', 'institut', 'groupe de recherche', 'personne', 'maladie']

ngrammes_kwic = [" ".join([t[0] for t in ng]).replace("' ", "'").replace("( ", "(").replace(" )", ")") for ng in ngrammes]

In [ ]:
ngrammes_kwic = [t for t in ngrammes_kwic if not 'STOP' in t]

In [ ]:
extrant = DataFrame(columns=['Mot-clé','Concordance', 'Fréquence (TF)'])
kwic = {w : [] for w in kw} 

In [ ]:
for t in ngrammes_kwic: # on pourrait aussi chercher dans les terms, mais on perd certains termes d'intérêt avec le filtrage syntaxique
    for w in kw:
        if t.startswith(w):
            kwic[w].append(t)

In [ ]:
kwic = {term: FreqDist(kwic[term]) for term in kwic}

In [ ]:
for term in kw:
    df_kw = DataFrame(kwic[term].items(), columns=['Concordance', "Fréquence (TF)"])
    df_kw.sort_values(["Fréquence (TF)"], 
        axis=0,
        ascending=[False], 
        inplace=True)

    df_kw.insert(0, 'Mot-clé', term)
    extrant = concat([extrant, df])


extrant = extrant[extrant['Fréquence (TF)'] > 30] 

# file_path = '../04-filtrage/output/'
# file_path = path.join(file_path, acteur, tag)


# extrant.to_csv(file_path + '_KWIC' +'.csv')

extrant

,Mot-clé,Concordance,Fréquence (TF),Terme,Structure syntaxique,LLR,p-value,Fréquence documentaire (DF)
0,NaN,NaN,501,directive,NOM,-,-,40.0
1,NaN,NaN,470,dose,NOM,-,-,16.0
2,NaN,NaN,385,vaccin,NOM,-,-,40.0
3,NaN,NaN,252,vaccination,NOM,-,-,39.0
4,NaN,NaN,243,santé,NOM,-,-,43.0
...,...,...,...,...,...,...,...,...
112,NaN,NaN,31,établissements,NOM,-,-,7.0
113,NaN,NaN,31,campagne de vaccination,NOM PRP NOM,342.4688,0.0,17.0
114,NaN,NaN,31,jan covid-19,NOM ADJ,221.069799,0.0,1.0
115,NaN,NaN,31,évaluation,NOM,-,-,11.0


### **Extraction de termes MeSH**

In [ ]:
df['isMeSHTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,directive,NOM,501,-,-,40,False
1,dose,NOM,470,-,-,16,False
2,vaccin,NOM,385,-,-,40,False
3,vaccination,NOM,252,-,-,39,False
4,santé,NOM,243,-,-,43,False
...,...,...,...,...,...,...,...
248,rsss,NOM,16,-,-,9,False
247,sodium,NOM,16,-,-,3,False
246,fréquence,NOM,16,-,-,9,False
245,trajectoire,NOM,16,-,-,12,False


In [ ]:
from nltk.tokenize import MWETokenizer
file_path = '../04-filtrage/MeSH/mesh-fr.txt'

with open (file_path, 'r', encoding='utf-8') as f:
    mesh = [tuple(tokenizer_re.tokenize(w)) for w in f.readlines()]
    tokenizer_mesh = MWETokenizer(mesh, separator= ' ')
    mesh = [tokenizer_mesh.tokenize(w)[0].lower() for w in mesh]
    mesh = [w for w in mesh if len(w.split()) > 1] # On ne retient que les termes complexes
    #mesh = [tuple(t.strip('.').lower().split()) for t in f.readlines()]

In [ ]:
extr_mesh = tokenizer_mesh.tokenize(list_terms)

In [ ]:
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,directive,NOM,501,-,-,40,False
1,dose,NOM,470,-,-,16,False
2,vaccin,NOM,385,-,-,40,False
3,vaccination,NOM,252,-,-,39,False
4,santé,NOM,243,-,-,43,False
...,...,...,...,...,...,...,...
248,rsss,NOM,16,-,-,9,False
247,sodium,NOM,16,-,-,3,False
246,fréquence,NOM,16,-,-,9,False
245,trajectoire,NOM,16,-,-,12,False


In [ ]:
for t in extr_mesh:
    if t in mesh:
        df.loc[df['Terme'] == t, 'isMeSHTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
0,directive,NOM,501,-,-,40,False
1,dose,NOM,470,-,-,16,False
2,vaccin,NOM,385,-,-,40,False
3,vaccination,NOM,252,-,-,39,False
4,santé,NOM,243,-,-,43,False
...,...,...,...,...,...,...,...
248,rsss,NOM,16,-,-,9,False
247,sodium,NOM,16,-,-,3,False
246,fréquence,NOM,16,-,-,9,False
245,trajectoire,NOM,16,-,-,12,False


In [ ]:
df[df['isMeSHTerm'] == 'True']

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm
44,services sociaux,NOM ADJ,61,752.299071,0.0,21,True
55,santé publique,NOM ADJ,51,533.781629,0.0,23,True
158,professionnels de la santé,NOM PRP DET:ART NOM,24,266.256636,0.0,14,True
222,santé mentale,NOM ADJ,18,199.222453,0.0,5,True


### **Extraction de termes existant dans la taxonomie**

In [ ]:
df['isTaxoTerm']= 'False' # On set à False puis on va changer pour True si on trouve le terme
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,directive,NOM,501,-,-,40,False,False
1,dose,NOM,470,-,-,16,False,False
2,vaccin,NOM,385,-,-,40,False,False
3,vaccination,NOM,252,-,-,39,False,False
4,santé,NOM,243,-,-,43,False,False
...,...,...,...,...,...,...,...,...
248,rsss,NOM,16,-,-,9,False,False
247,sodium,NOM,16,-,-,3,False,False
246,fréquence,NOM,16,-,-,9,False,False
245,trajectoire,NOM,16,-,-,12,False,False


In [ ]:
file_path = '../04-filtrage/default_taxo_labels.csv'

with open(file_path, 'r', encoding='utf-8') as f:
    default = read_csv(f, sep=';')
    labels = list(dict.fromkeys([str(t).strip().lower() for t in default['Label'].tolist()]))

labels

['1 on 1 support',
 '10 weeks and less ultrasound',
 '12-step facilitation therapy',
 '12-step programs',
 '18f pet bone scan',
 '18f bone pet',
 '18f bone scan',
 '1st generation antipsychotics',
 '2019 ncov testing',
 '2d doppler echocardiography',
 '2d echocardiography',
 '2d ultrasound',
 '2nd generation antipsychotics',
 '3d imaging',
 '3d sonography',
 '3d dental imaging',
 '3d echocardiography',
 '3d fetal ultrasound',
 '3d mammography',
 '3d prenatal ultrasound',
 '3d reconstruction',
 '46, xx disorder of sex development',
 '46,xy disorder of sex development',
 '4d echocardiography',
 '4d prenatal ultrasound',
 'a1c diabetes test',
 'a1c test',
 'abo + rh pnl bld',
 'abo and rh group in cord blood',
 'abpm device',
 'acl reconstruction surgery',
 'acth deficiency',
 'acth stimulation test',
 'adhd assessment',
 'adhd diagnostic',
 'adhd evaluation',
 'adhd testing',
 'adn foetal circulant',
 'aids',
 'aids nephropathy',
 'aids serodiagnosis',
 'aids-associated nephropathy',
 'a

In [ ]:
for t in df['Terme'].tolist():
    if t in labels:
        df.loc[df['Terme'] == t, 'isTaxoTerm'] = 'True'
df

,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
0,directive,NOM,501,-,-,40,False,False
1,dose,NOM,470,-,-,16,False,False
2,vaccin,NOM,385,-,-,40,False,True
3,vaccination,NOM,252,-,-,39,False,True
4,santé,NOM,243,-,-,43,False,False
...,...,...,...,...,...,...,...,...
248,rsss,NOM,16,-,-,9,False,False
247,sodium,NOM,16,-,-,3,False,False
246,fréquence,NOM,16,-,-,9,False,False
245,trajectoire,NOM,16,-,-,12,False,False


In [ ]:
# Par exemple, on peut extraire les termes MeSH qui ne sont pas dans la taxonomie, mais qui se trouvent dans le corpus d'intérêt
df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()

C:\Users\p1115145\AppData\Local\Temp\ipykernel_10512\4006263597.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df[df['isTaxoTerm'] == 'False'][df['isMeSHTerm'] == 'True'] #['Terme'].tolist()


,Terme,Structure syntaxique,Fréquence (TF),LLR,p-value,Fréquence documentaire (DF),isMeSHTerm,isTaxoTerm
158,professionnels de la santé,NOM PRP DET:ART NOM,24,266.256636,0.0,14,True,False


In [ ]:
if lng == 'en':
    acteur = acteur + '_' + lng
if tag:
    output_path = path.join('../04-filtrage/output/', acteur + '_' + tag + '_significant-collocations.csv') 
else:
    output_path = path.join('../04-filtrage/output/', acteur + '_significant-collocations.csv') 
df.to_csv(output_path)